Suppose the spam generator becomes more intelligent and begins producing prose which looks "more legitimate" than before. 

There are numerous ways the prose could become more like legitimate text. For the purpose of this notebook we will simply force the spam data to 'drift' by adding the first few lines of Pride and Prejudice to the start of the spam documents in our testing set. We will then see how the trained model responds.  

In [1]:
import pandas as pd
df = pd.read_parquet("data/training.parquet")

We split the data into training and testing sets, as in the modelling notebooks. We use the 'random_state' parameter to ensure that the data is split in the same way as it was when we fit the model. 

In [2]:
from sklearn import model_selection

df_train, df_test = model_selection.train_test_split(df, random_state=43)
df_test_spam = df_test[df_test.label == 'spam'].copy() #filter the spam documents

In [3]:
def add_text(doc, adds):
    """
    takes in a string _doc_ and
    appends text _adds_ to the start
    """
    
    return adds + doc

In [4]:
pride_pred = '''It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?” Mr. Bennet replied that he had not. “But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.” Mr. Bennet made no answer. “Do you not want to know who has taken it?” cried his wife impatiently.'''

In [5]:
# appending text to the start of the spam
df_test_spam["text"] = df_test_spam.text.apply(add_text, adds=pride_pred)

In [6]:
pd.set_option('display.max_colwidth', -1) #ensures that all the text is visible
df_test_spam.sample(3)

,index,label,text
31699,11699,spam,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?” Mr. Bennet replied that he had not. “But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.” Mr. Bennet made no answer. “Do you not want to know who has taken it?” cried his wife impatiently.I have two dogs ... a little 15 pound pug a poo and a 90 pound native american indian dog. 5gs of fat. I paid for all the lovers of this cereal and want it back for a second order. Along with the ease of use. Secondly- this stuff is because it's not worth it in my Blue Willow Teapot! This one puts the bitterness on display, to good effect. My husband and I did not realize the quantity was extremely misleading, akin to a lovely day while your supply lasts."
35509,15509,spam,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?” Mr. Bennet replied that he had not. “But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.” Mr. Bennet made no answer. “Do you not want to know who has taken it?” cried his wife impatiently.This product is delishy. These are so delicious! I can only recommend this as a Christmas gift right before Christmas. Too much is more than enough time. My mother always just fed them whatever, the kinds of food you buy in the supermarket at the checkout counter at WalMart for much less then this, plus better ingredients!"
24617,4617,spam,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?” Mr. Bennet replied that he had not. “But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.” Mr. Bennet made no answer. “Do you not want to know who has taken it?” cried his wife impatiently.My suggestions is to try this formula, saying give it 2 weeks to try again and hope that it sells- sorry I tried this tea. My girl loves dirty martinis. Now I think I will invest in a lock box to keep my lips moisturized; also works for iced coffee. I will be coming back for more. I added orange juice and it really seems to love them for the best price. My kids bring it when they come out with many other waters. It is a great value for the product. I have been buying Bones from Amazon, and, only Booda Bones, ever since."


We now pass this `drifted` data through the pipeline we created: we compute feature vectors, and we make spam/legitimate classifications using the model we trained. 

In [7]:
from sklearn.pipeline import Pipeline
import pickle, os

## loading in feature vectors pipeline
import pickle, os
filename = 'feature_pipeline.sav'
feat_pipeline = pickle.load(open(filename, 'rb'))

## loading model
filename = 'model.sav'
model = pickle.load(open(filename, 'rb'))



/anaconda2/envs/mlworkflow/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [10]:
pipeline = Pipeline([
    ('features',feat_pipeline),
    ('model',model)
])

## we need to fit the model, using the un-drifted data, as we did in the previous notebooks. 

pipeline.fit(df_train["text"], df_train["label"])



Pipeline(memory=None,
     steps=[('features', Pipeline(memory=None,
     steps=[('features', <mlworkflows.featuressimple.SimpleSummaries object at 0x1a1ec80438>)])), ('model', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
    ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [11]:
## we can then go on and make predictions for the drifted spam, using the fitted pipeline above. 
# predict test instances
y_preds = pipeline.predict(df_test_spam["text"])
print(y_preds)

['legitimate' 'legitimate' 'legitimate' ... 'legitimate' 'legitimate'
 'spam']


In [13]:
import numpy as np
np.array(np.unique(y_preds, return_counts = True))

array([['legitimate', 'spam'],
       [4235, 761]], dtype=object)

The model is worse at classifying drifted data, since this is not what we trained the model on. 

## Exercises
The two models perform very similarly on the 'drifted' data in this notebook. Consider alternative types of data drift and see how the models perform: 
1. What happens when fewer words from Pride and Prejudice are appended to the spam? 
2. How about using a completely different exert of Austen? 
3. How do the models perform when generic text (neither Austen nor food reviews) is appended to the spam? 